In [1]:
import pandas as pd
import numpy as np

# READ THE CSV INTO DATAFRAME

df = pd.read_csv('Syngenta/Syngenta_2017/Experiment_dataset.csv')
# df2 = pd.read_csv('Syngenta/Syngenta_2017/Region_dataset.csv')

In [2]:
# CURRENTLY NECESSARY IF: USING 174 ADDITIONAL VARIETY COLUMNS METHOD

# THIS IS A DIFFERENT APPROACH TO THE ABOVE FOUR CELLS, WHERE WE HAVE 174 ADDITIONAL FEATURE COLUMNS
# EACH WITH A 0 (IF IT IS NOT OF THAT VARIETY) OR A 1 (IF IT IS OF THAT VARIETY)

# print(df)
dummies = pd.get_dummies(df.Variety)
# print(dummies)
df = pd.concat([df, dummies], axis=1)
print(df)

        Experiment  Location  Variety Planting date  Yield  Check Yield  \
0      09SUBWYG201      2291  V000047       5/20/09  55.62        53.61   
1      09SUBWYG202      2291  V000047       5/20/09  53.72        53.61   
2      09SUBWYG203      2291  V000047       5/20/09  49.09        53.61   
3      09SUBWYG204      2291  V000047       5/20/09  53.00        53.61   
4      09SUBWYG301      2290  V130307        5/8/09  44.51        57.32   
5      09SUBWYG301      2290  V000025        5/8/09  47.41        57.32   
6      09SUBWYG302      2290  V000024        5/8/09  49.15        57.32   
7      09SUBWYG302      2290  V130307        5/8/09  43.23        57.32   
8      09SUBWYG302      2290  V000124        5/8/09  50.99        57.32   
9      09SUBWYG302      2290  V000025        5/8/09  51.32        57.32   
10     09SUBWYG302      3490  V000024       5/26/09  70.99        61.90   
11     09SUBWYG302      3490  V130307       5/26/09  65.61        61.90   
12     09SUBWYG302      3

In [3]:
# DROP ALL THE CELLS THAT ARE NOT USABLE SUCH AS THE ONES THAT ARE STRINGS OR DATES

df = df.drop(['Experiment', 'Location', 'Planting date', 'Check Yield', 'Yield difference', 'Latitude', 'Longitude', 'Variety', 'PI'], axis=1)
df['YieldBucket'] = pd.Series(pd.qcut(df.Yield, q=3, labels=["high", "medium", "low"]))


In [4]:
# LET US ALSO MAKE SURE THERE ARE NO NAN IN THE DATA
print("We expect to be %s nan values and there actually are %s nan values\n" % (0, np.sum(df.isnull().sum())))
# AFTER COLUMNS, MAKE SURE NO SKETCHY ONES
for col in df.columns:
    print(col, type(df[col][0]))    

We expect to be 0 nan values and there actually are 0 nan values

Yield <class 'numpy.float64'>
Year <class 'numpy.int64'>
Temperature <class 'numpy.float64'>
Precipitation <class 'numpy.float64'>
Solar Radiation <class 'numpy.int64'>
Soil class <class 'numpy.int64'>
CEC <class 'numpy.float64'>
Organic matter <class 'numpy.float64'>
pH <class 'numpy.float64'>
Clay <class 'numpy.float64'>
Silt <class 'numpy.float64'>
Sand <class 'numpy.float64'>
Area <class 'numpy.float64'>
V000016 <class 'numpy.uint8'>
V000017 <class 'numpy.uint8'>
V000018 <class 'numpy.uint8'>
V000023 <class 'numpy.uint8'>
V000024 <class 'numpy.uint8'>
V000025 <class 'numpy.uint8'>
V000030 <class 'numpy.uint8'>
V000032 <class 'numpy.uint8'>
V000034 <class 'numpy.uint8'>
V000036 <class 'numpy.uint8'>
V000039 <class 'numpy.uint8'>
V000047 <class 'numpy.uint8'>
V000050 <class 'numpy.uint8'>
V000051 <class 'numpy.uint8'>
V000058 <class 'numpy.uint8'>
V000060 <class 'numpy.uint8'>
V000062 <class 'numpy.uint8'>
V000067 <cla

In [5]:
# TRAIN AND TEST SPLIT# TRAIN AND TEST SPLIT# TRAIN AND TEST SPLIT
# TRAIN AND TEST SPLIT# TRAIN AND TEST SPLIT# TRAIN AND TEST SPLIT
# TRAIN AND TEST SPLIT# TRAIN AND TEST SPLIT# TRAIN AND TEST SPLIT
# TRAIN AND TEST SPLIT# TRAIN AND TEST SPLIT# TRAIN AND TEST SPLIT

# feature_columns = ['Variety', 'Solar Radiation', 'Temperature', 'Precipitation', 'Location', 'Clay', 'Silt', 'Sand', 'pH', 'Precipitation', 'CEC', 'Soil class']

X = df.drop(['Yield', 'YieldBucket'], axis=1)

print(X.columns)
# X = df.loc[:, feature_columns]

y = df.Yield

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.05, train_size = 0.1, random_state = 42)

# train_visual, zz, zzz, zzzz = train_test_split(df, y, test_size=0.2, train_size=0.8, random_state = 42)

from sklearn import preprocessing
from sklearn import utils

Index(['Year', 'Temperature', 'Precipitation', 'Solar Radiation', 'Soil class',
       'CEC', 'Organic matter', 'pH', 'Clay', 'Silt',
       ...
       'V156565', 'V156574', 'V156642', 'V156763', 'V156774', 'V156783',
       'V156786', 'V156797', 'V156806', 'V156807'],
      dtype='object', length=186)


In [8]:
from sklearn.metrics import mean_squared_error
def evaluate_errors(prediction, actual):
    print("RMSE Error: ", np.sqrt(mean_squared_error(prediction, actual)))
    avg_error_vector = np.absolute(((preds - y_test) / y_test) * 100))
    print("Average Error: ", np.mean(avg_error_vector))
    print("Average Error std: ", avg_error_vector.describe())

SyntaxError: invalid syntax (<ipython-input-8-dbf4d5c533b7>, line 4)

In [6]:
from sklearn.ensemble import RandomForestRegressor
regr = RandomForestRegressor(n_estimators=10, max_depth=20, random_state=0, verbose=1)
regr.fit(X_train, y_train)
preds = regr.predict(X_test)

from sklearn.metrics import mean_squared_error

mse = mean_squared_error(preds, y_test)

errors = np.absolute(((preds - y_test) / y_test) * 100)
# erros = 
# print(errors)
print("mse:", mse)
print(np.mean(errors))

mse: 53.13332545359653
10.005838207423524


[Parallel(n_jobs=1)]: Done  10 out of  10 | elapsed:    0.6s finished
[Parallel(n_jobs=1)]: Done  10 out of  10 | elapsed:    0.0s finished


In [7]:
# GET OUTPUT OF FEATURE IMPORTANCE

feature_importances = regr.feature_importances_
feature_importances = pd.Series(feature_importances)
feature_importance_df = pd.DataFrame({'feature': X_train.columns,'feature_importance': feature_importances})
feature_importance_df = feature_importance_df.sort_values(by=['feature_importance'])
for index, row in feature_importance_df.iterrows():
    print(row['feature'], 'has importance: ', row['feature_importance'])
# for feature_importance in regr.feature_importances_:
    
print(errors.describe())

V000030 has importance:  0.0
V140784 has importance:  1.206935965562934e-05
V110923 has importance:  1.5094549292347343e-05
V151340 has importance:  2.3597708000294508e-05
V151331 has importance:  3.685303763063591e-05
V103308 has importance:  3.805664579449813e-05
V137289 has importance:  3.8592331805538116e-05
V155180 has importance:  5.005169316426522e-05
V152779 has importance:  5.354413365757188e-05
V139094 has importance:  5.6237778946373004e-05
V000017 has importance:  6.27706373232473e-05
V121524 has importance:  6.619417827328127e-05
V103332 has importance:  6.681548513345065e-05
V156783 has importance:  6.906792443807952e-05
V114944 has importance:  9.306059238104696e-05
V151336 has importance:  9.317815692431577e-05
V000036 has importance:  9.320209313144743e-05
V113424 has importance:  9.772856666629215e-05
V000070 has importance:  0.00010775266059769694
V031243 has importance:  0.00011526433297735909
V120585 has importance:  0.00011959113313673203
V103193 has importance:  

In [8]:
# THIS WILL ONLY WORK WITH THE BUCKET METHOD

from sklearn.ensemble import RandomForestClassifier
regr = RandomForestClassifier(n_estimators=10, max_depth=20, random_state=0, verbose=1)
regr.fit(X_train, y_train)
preds = regr.predict(X_test)

from sklearn.metrics import accuracy_score

print(accuracy_score(y_test, preds))

ValueError: Unknown label type: 'continuous'

In [9]:
import numpy as np
from sklearn import linear_model
from sklearn import svm
from sklearn.metrics import accuracy_score
from sklearn.neural_network import MLPRegressor

from sklearn.feature_selection import RFECV

classifiers = [
    svm.SVR(),
    MLPRegressor(solver='lbfgs', alpha=1e-5,
                     hidden_layer_sizes=(5, 2), random_state=1),
    linear_model.SGDRegressor(),
    linear_model.BayesianRidge(),
    linear_model.LassoLars(),
#     linear_model.ARDRegression(),
#     linear_model.ARDRegression(),
    linear_model.PassiveAggressiveRegressor(),
    linear_model.TheilSenRegressor(),
    linear_model.LinearRegression()]




# estimator = svm.SVR(kernel="linear")

# selector = RFECV(estimator, step=1, cv=5, verbose=1)
# selector = selector.fit(X_train, y_train)
# selector.support_ 
# # array([ True,  True,  True,  True,  True,
# #         False, False, False, False, False], dtype=bool)
# selector.ranking_
# # array([1, 1, 1, 1, 1, 6, 4, 3, 2, 5])


#     print(np.sum(preds - y_test))
#     print(clf.predict(X_test),'\n')
#     print(y_test)
#     print('accuracy score:', accuracy_score(y_test, clf.predict(X_test)), '\n')


In [ ]:
for item in classifiers:
    print(item)
    clf = item
    clf.fit(X_train, y_train)
    preds = clf.predict(X_test)
    errors = mean_squared_error(preds, y_test)
#     errors = np.absolute(((preds - y_test) / y_test) * 100)
#     print(errors)
    print(errors)
#     print(errors.describe())

SVR(C=1.0, cache_size=200, coef0=0.0, degree=3, epsilon=0.1, gamma='auto',
  kernel='rbf', max_iter=-1, shrinking=True, tol=0.001, verbose=False)
67.91605839333502
MLPRegressor(activation='relu', alpha=1e-05, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(5, 2), learning_rate='constant',
       learning_rate_init=0.001, max_iter=200, momentum=0.9,
       nesterovs_momentum=True, power_t=0.5, random_state=1, shuffle=True,
       solver='lbfgs', tol=0.0001, validation_fraction=0.1, verbose=False,
       warm_start=False)
132.20461002435297
SGDRegressor(alpha=0.0001, average=False, epsilon=0.1, eta0=0.01,
       fit_intercept=True, l1_ratio=0.15, learning_rate='invscaling',
       loss='squared_loss', max_iter=None, n_iter=None, penalty='l2',
       power_t=0.25, random_state=None, shuffle=True, tol=None, verbose=0,
       warm_start=False)
3.536828786322753e+41
BayesianRidge(alpha_1=1e-06, alpha_2=1e-06, compute_score=F

/home/kafi/.local/lib/python3.5/site-packages/sklearn/linear_model/stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.stochastic_gradient.SGDRegressor'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)


110.77074625717506
LassoLars(alpha=1.0, copy_X=True, eps=2.220446049250313e-16,
     fit_intercept=True, fit_path=True, max_iter=500, normalize=True,
     positive=False, precompute='auto', verbose=False)
132.2046100236892
ARDRegression(alpha_1=1e-06, alpha_2=1e-06, compute_score=False, copy_X=True,
       fit_intercept=True, lambda_1=1e-06, lambda_2=1e-06, n_iter=300,
       normalize=False, threshold_lambda=10000.0, tol=0.001, verbose=False)


KeyboardInterrupt: 

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.datasets import make_moons, make_circles, make_classification
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis

classifiers = [
#     KNeighborsClassifier(3),
#     SVC(kernel="linear", C=0.025),
    SVC(gamma=2, C=1),
#     GaussianProcessClassifier(1.0 * RBF(1.0)),
#     DecisionTreeClassifier(max_depth=5),
    RandomForestClassifier(max_depth=5, n_estimators=10, max_features=1),
    MLPClassifier(alpha=1),
    AdaBoostClassifier(),
    GaussianNB(),
    QuadraticDiscriminantAnalysis()]
from sklearn.metrics import accuracy_score
for item in classifiers:
    print(item)
    clf = item
    clf.fit(scale(X_train), y_train)
    preds = clf.predict(scale(X_test))
    print(accuracy_score(y_test, preds))
#     errors = np.absolute(((preds - y_test) / y_test) * 100)
#     print(errors)
#     print(np.mean(errors))